In [1]:
import cobra
import pandas as pd
import requests

In [2]:
covid_cobra = cobra.io.read_sbml_model('../Models/iAB_AMO1410_SARS-CoV-2.xml')

Using license file /usr/local/gurobi/gurobi.lic
Set parameter TokenServer to value sccsvc


In [3]:
solution = covid_cobra.optimize()

In [4]:
#How we come out with subsystems for the reactions

#Special Reactions I classified manually (listed in dictionary format)
#These were reactions that differed between the lipids and no lipids models
invalidFile = open('InvalidReactions.txt')
invalid = eval(invalidFile.read())

#Output file containing reach reaction's subsystem in the format of
#<Index> <Subsystem>
output = open('ReactionLabels.txt', 'w')

idx = 0
for i in solution.fluxes.index:
    
    #Unknown classification as default
    subsystem = 'Unknown'
    
    try:
        
        #If 'EX' is in the name, it signifies an exchange reaction
        if ('EX' in i): 
            subsystem = 'Extracellular exchange'
            f2.write(str(idx) + ' ' + subsystem + '\n')
            idx += 1
            continue
        
        #Try to access BiGG's subsystem
        r = requests.get('http://bigg.ucsd.edu/api/v2/models/RECON1/reactions/' + i)
        subsystem = r.json()['results'][0]['subsystem']
        
    except:
        #If BiGG does not have the subsystem, check if I manually classified them
        if (i in invalid): subsystem = invalid[i]
                
    output.write(str(idx) + ' ' + subsystem + '\n')
    idx += 1

In [5]:
output.flush() #Flush the buffer so it actually writes to the file